In [3]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline 

In [3]:
# Imports
from pyspark import SparkConf, SparkContext
from sklearn.neighbors import NearestNeighbors
from pyspark.mllib.regression import LabeledPoint


#Creating RDD that contains all rows of CSV file except Header
csvf=sc.textFile("CategoryNormalized.csv")\
        .filter(lambda x: 'prev' not in x)\
        .map(lambda x:x.encode('ascii').split(','))
        #.map(Convert)

        
# Creating Header using first line of csv file
Header=sc.textFile("CategoryNormalized.csv").map(lambda x:x.encode('ascii').split(',')).first()
#csvf.take(3)
CV_data = sqlCtx.createDataFrame(csvf,Header)

In [4]:
def labelData(data):
    data=data.rdd
    # label: row[end], features: row[0:end-1]
    return data.map(lambda row: LabeledPoint(row[-1], row[0:-1]))

training_data, testing_data = labelData(CV_data).randomSplit([0.8, 0.2])

In [20]:
#We are calling labeldata function that is defined in decision tree model that will submit data for percentage split
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
#RFdata = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')


def labelData(data):
    data=data.rdd
    # label: row[end], features: row[0:end-1]
    return data.map(lambda row: LabeledPoint(row[-1], row[0:-1]))


# Split the data into training and test sets (20% held out for testing)

RFtraining_data, RFtesting_data = labelData(CV_data).randomSplit([0.8, 0.2])



In [22]:
#RandomForest Model training
RFmodel = RandomForest.trainClassifier(RFtraining_data, numClasses=3, categoricalFeaturesInfo={},
                                     numTrees=15, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=17, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = RFmodel.predict(RFtesting_data.map(lambda x: x.features))

labelsAndPredictions = RFtesting_data.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(RFtesting_data.count())
#print('Accuracy of Test = ' + str(1-testErr))

Trainpredictions = RFmodel.predict(RFtraining_data.map(lambda x: x.features))
TrlabelsAndPredictions = RFtraining_data.map(lambda lp: lp.label).zip(Trainpredictions)
trainErr = TrlabelsAndPredictions.filter(lambda (v, p): v != p).count() / float(RFtraining_data.count())

labelsAndPredictions

print('Accuracy of Train = ' + str(1-trainErr))
print('Accuracy of Test = ' + str(1-testErr))

Accuracy of Train = 0.941921363246
Accuracy of Test = 0.877867002572


In [47]:
from pyspark.mllib.evaluation import MulticlassMetrics

def getPredictionsLabels(RFmodel, RFtesting_data):
    predictions = RFmodel.predict(RFtesting_data.map(lambda r: r.features))
    return predictions.zip(RFtesting_data.map(lambda r: r.label))

def printMetrics(predictions_and_labels):
    metrics = MulticlassMetrics(predictions_and_labels)
    print 'F-1 Score         ', metrics.fMeasure()
    #print 'F Score         ', metrics.fMeasureByThreshold()
    print 'Confusion Matrix\n', metrics.confusionMatrix().toArray()

#Printing F1 Score and Confusion Matrix
print('Training Set')
training_labels = getPredictionsLabels(RFmodel, RFtraining_data)
printMetrics(training_labels)

print('\n')

print ('Test Set')
predictions_and_labels = getPredictionsLabels(RFmodel, RFtesting_data)
printMetrics(predictions_and_labels)

Training Set
F-1 Score          0.941921363246
Confusion Matrix
[[ 23353.    954.    433.]
 [   710.  26112.   1397.]
 [   166.    798.  22835.]]


Test Set
F-1 Score          0.877867002572
Confusion Matrix
[[ 5508.   466.   132.]
 [  558.  5729.   746.]
 [   91.   334.  5489.]]


In [46]:

#                       Decision Tree Classifier Starts Here
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

def labelData(data):
    data=data.rdd
    # label: row[end], features: row[0:end-1]
    return data.map(lambda row: LabeledPoint(row[-1], row[:-1]))

#Splitting the Data into Training and Test Set
training_data, testing_data = labelData(CV_data).randomSplit([0.8, 0.2])

#Building The Decision tree Classifier Model
model = DecisionTree.trainClassifier(training_data, numClasses=3, maxDepth=17,
                                     categoricalFeaturesInfo={},
                                     impurity='gini', maxBins=32)

from pyspark.mllib.evaluation import MulticlassMetrics

def getPredictionsLabels(model, test_data):
    predictions = model.predict(test_data.map(lambda r: r.features))
    return predictions.zip(test_data.map(lambda r: r.label))

def printMetrics(predictions_and_labels):
    metrics = MulticlassMetrics(predictions_and_labels)
    print 'F-1 Score         ', metrics.fMeasure()
    #print 'F Score         ', metrics.fMeasureByThreshold()
    print 'Confusion Matrix\n', metrics.confusionMatrix().toArray()

#Printing F1 Score and Confusion Matrix
print('Training Set')
training_labels = getPredictionsLabels(model, training_data)
printMetrics(training_labels)

print('\n')

print ('Test Set')
predictions_and_labels = getPredictionsLabels(model, testing_data)
printMetrics(predictions_and_labels)


Training Set
F-1 Score          0.894007471069
Confusion Matrix
[[ 22791.   1086.    831.]
 [  1837.  23671.   2530.]
 [   540.   1291.  21985.]]


Test Set
F-1 Score          0.843524338927
Confusion Matrix
[[ 5368.   516.   254.]
 [  822.  5531.   861.]
 [  143.   416.  5338.]]
